In [ ]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib

# Importamos las librerías necesarias
import numpy as np
import cv2 as cv
import time
import matplotlib.pyplot as plt

In [ ]:
# Puntero a la webcam
# -------------------
cap = cv.VideoCapture(0)
time.sleep(1) # Para garantizar la correcta inicializacion de la webcam

# Tomamos el primer frame del video (donde vamos a definir la ROI)
#-----------------------------------------------------------------
ret,frame = cap.read()
cv.imwrite('Frame1.jpg',frame)
plt.figure('frame')
plt.imshow(frame)

# Obtenemos la resolucion de la imagen
# ------------------------------------
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

In [ ]:
# Seleccionamos la ROI con el mouse, presionando enter al finalizar
# -----------------------------------------------------------------
ROI = cv.selectROI('frame',frame, False, False)
x, y, w, h = ROI
track_window = (x, y, w, h)

In [ ]:
# Una vez definida la ROI recortamos esa parte de la imagen que utilizaremos para seguimiento
#--------------------------------------------------------------------------------------------
roi = frame[y:y+h, x:x+w]
# Pasamos la ROI a HSV (los valores de H son más estables a cambios de intensidad)
hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)
# Definimos una máscara para el histograma (Hue: 0..180, Saturation:60..255, Value:32..255)
mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
# Y calculamos el histograma sobre esa máscara (toma solo el Hue: tonalidad)
roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
# Normalizamos el histograma (entre 0 y 255 según indica el flag cv.NORM_MINMAX)
cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)

# Mostramos lo que se fue armando en relación a la ROI
#-----------------------------------------------------
plt.figure()
plt.imshow(mask)

In [ ]:
# Condición de terminación: 10 iteraciones o moverse al menos 1pt
#----------------------------------------------------------------
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 1, 10 )

# Elegimos la duracion del video a capturar
#------------------------------------------
timeout = 40 # segundos
timeout_start = time.time()

# Definimos el archivo de salida del video
# ----------------------------------------
out = cv.VideoWriter('Output.mp4',cv.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

while time.time() < timeout_start + timeout:
    ret, frame = cap.read()
    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        # Aplicamos camshift al nuevo frame
        #----------------------------------
        ret, track_window = cv.CamShift(dst, track_window, term_crit)
        
        
        # Con escalamiento de ventana - Sin rotación
        #-------------------------------------------
        x,y,w,h = track_window
        img2 = cv.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        
        out.write(img2)
        cv.imshow('Seguimiento',img2)
        
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
cv.destroyAllWindows()
cap.release()